In [2]:
from pathlib import Path
import os
from IPython.display import HTML
from mp_transformer.config import CONFIG
from mp_transformer.train import setup
from mp_transformer.utils import save_side_by_side_video

In [3]:
current_dir = Path.cwd().parts[-1]
if current_dir == "demo":
    os.chdir("..")
current_dir = Path.cwd().parts[-1]
!pwd
if current_dir != "mp-transformer":
    raise ValueError("Please run this notebook in the project root.")

/data/daniel/git/mp-transformer
